# VacationPy
---
## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,st. john's,47.5649,-52.7093,0.93,94,75,13.38,CA,1708097402
1,1,baukau,51.5526,7.2042,13.92,93,100,2.63,DE,1708097677
2,2,itapuranga,-15.5622,-49.9486,28.23,62,92,1.16,BR,1708097677
3,3,waitangi,-43.9535,-176.5597,15.46,92,77,0.89,NZ,1708097677
4,4,miquelon,47.0975,-56.3814,-1.00,77,100,13.66,PM,1708097677


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City"
    )

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
city_data_df = city_data_df.dropna(how="any")

# vaction loction perference sunny, not too hot, not cold :)
reduce_df = city_data_df.loc[(city_data_df["Cloudiness"] <= 10) & (city_data_df["Max Temp"] <= 26) & (city_data_df["Max Temp"] >= 23), :]

# Display sample data
reduce_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,dolores,-36.3132,-57.6792,25.72,53,0,5.09,AR,1708097680
64,64,howrah,22.5892,88.3103,24.00,73,0,2.06,IN,1708097684
99,99,east london,-33.0153,27.9116,24.52,46,0,4.12,ZA,1708097693
191,191,pucon,-39.2822,-71.9543,24.27,48,1,1.71,CL,1708097707
217,217,kone,-21.0595,164.8658,24.33,86,4,2.55,NC,1708097711
248,248,dakhla,23.6848,-15.9580,23.01,60,0,11.83,EH,1708097716
255,255,port elizabeth,-33.9180,25.5701,23.16,53,0,12.35,ZA,1708097717
258,258,sur,22.5667,59.5289,23.75,50,3,3.36,OM,1708097718
263,263,queenstown,-31.8976,26.8753,23.17,12,3,8.52,ZA,1708097718
279,279,khasab,26.1799,56.2477,24.55,69,6,1.49,OM,1708097720


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = reduce_df.loc[:,["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
26,dolores,AR,-36.3132,-57.6792,53,
64,howrah,IN,22.5892,88.3103,73,
99,east london,ZA,-33.0153,27.9116,46,
191,pucon,CL,-39.2822,-71.9543,48,
217,kone,NC,-21.0595,164.8658,86,
248,dakhla,EH,23.6848,-15.9580,60,
255,port elizabeth,ZA,-33.9180,25.5701,53,
258,sur,OM,22.5667,59.5289,50,
263,queenstown,ZA,-31.8976,26.8753,12,
279,khasab,OM,26.1799,56.2477,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000

# Set up a parameters dictionary
params = {

 "categories":"accommodation.hotel",
 "limit":"10",
 "apiKey":geoapify_key

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty & Convert the API response to JSON format
    name_address = requests.get(base_url,params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"   
      
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
dolores - nearest hotel: Plaza Hotel
howrah - nearest hotel: Sun India Guest House
east london - nearest hotel: No hotel found
pucon - nearest hotel: No hotel found
kone - nearest hotel: Pacifik Appartels
dakhla - nearest hotel: Hotel El Massira
port elizabeth - nearest hotel: Waterford Hotel
sur - nearest hotel: Sur Hotel
queenstown - nearest hotel: Queens Hotel
khasab - nearest hotel: "Extra Divers Villa"
tadine - nearest hotel: Hôtel Nengoné
moog - nearest hotel: Lohas Hotel
mandideep - nearest hotel: Hotel Shiv Vilas 
necochea - nearest hotel: Cabañas Río Quequén
malvan - nearest hotel: Hotel Silver Sand
sultanah - nearest hotel: فندق جلنار


,City,Country,Lat,Lng,Humidity,Hotel Name
26,dolores,AR,-36.3132,-57.6792,53,Plaza Hotel
64,howrah,IN,22.5892,88.3103,73,Sun India Guest House
99,east london,ZA,-33.0153,27.9116,46,No hotel found
191,pucon,CL,-39.2822,-71.9543,48,No hotel found
217,kone,NC,-21.0595,164.8658,86,Pacifik Appartels
248,dakhla,EH,23.6848,-15.9580,60,Hotel El Massira
255,port elizabeth,ZA,-33.9180,25.5701,53,Waterford Hotel
258,sur,OM,22.5667,59.5289,50,Sur Hotel
263,queenstown,ZA,-31.8976,26.8753,12,Queens Hotel
279,khasab,OM,26.1799,56.2477,69,"""Extra Divers Villa"""


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    hover_cols = ["Hotel Name", "Country"],
    frame_width = 800,
    frame_height = 500,
    )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)